In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import joblib

In [5]:
#load data
df = pd.read_csv('../model_data.csv')

#drop auto generated column (no use)
df.drop(["Unnamed: 0"], axis=1, inplace = True)

#check for NA instances
print("# of na rows per column")
print(df.isna().sum())

#check for duplicates
print("# of duplicates")
print(df.duplicated().sum())
#df.drop_duplicates(inplace=True)

def determine_phase(df):
    if (df['Water Content(mol)'] != 0) & (df['contains_solid'] == 0):
        return 'liquid'
    elif (df['Water Content(mol)'] != 0) & (df['contains_solid'] == 1):
        return 'liquid/solid'
    else:
        return 'solid' 

df['phase'] = df.apply(determine_phase, axis=1)

print(df['phase'].value_counts())

df.describe()

# of na rows per column
Temp(K)               0
RH                    0
N_H+(mol)             0
N_NH4+(mol)           0
N_Na+(mol)            0
N_SO42-(mol)          0
N_NO3-(mol)           0
N_Cl-(mol)            0
N_OH-(mol)            0
N_NH3-(mol)           0
Water Content(mol)    0
P_HNO3(atm)           0
P_HCL(atm)            0
P_NH3(atm)            0
P_H2SO4(atm)          0
P_HBr(atm)            0
contains_solid        0
dtype: int64
# of duplicates
0
liquid/solid    11160
liquid           7390
solid            5950
Name: phase, dtype: int64


,Temp(K),RH,N_H+(mol),N_NH4+(mol),N_Na+(mol),N_SO42-(mol),N_NO3-(mol),N_Cl-(mol),N_OH-(mol),N_NH3-(mol),Water Content(mol),P_HNO3(atm),P_HCL(atm),P_NH3(atm),P_H2SO4(atm),P_HBr(atm),contains_solid
count,24500.00,24500.000000,24500.0,2.450000e+04,2.450000e+04,2.450000e+04,2.450000e+04,2.450000e+04,24500.0,24500.0,2.450000e+04,2.450000e+04,2.450000e+04,2.450000e+04,2.450000e+04,24500.0,24500.000000
mean,298.15,0.570000,0.0,4.559251e-08,1.953965e-08,7.335343e-09,1.628304e-08,3.419438e-08,0.0,0.0,1.865244e-07,8.957591e-12,3.929536e-11,9.944820e-07,5.107882e-29,0.0,0.698367
std,0.00,0.180558,0.0,1.448640e-08,7.577349e-09,4.396267e-09,8.369788e-09,1.079810e-08,0.0,0.0,2.382971e-07,1.242647e-11,4.166712e-11,6.985366e-07,3.626244e-29,0.0,0.458976
min,298.15,0.250000,0.0,2.324523e-08,7.748410e-09,0.000000e+00,3.874205e-09,1.743392e-08,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000
25%,298.15,0.450000,0.0,3.348078e-08,1.355972e-08,4.050000e-09,9.685512e-09,2.511059e-08,0.0,0.0,4.222870e-09,9.831530e-13,1.156060e-11,2.655840e-07,0.000000e+00,0.0,0.000000
50%,298.15,0.575000,0.0,4.319325e-08,1.811250e-08,6.657775e-09,1.488113e-08,3.264098e-08,0.0,0.0,6.812875e-08,5.477985e-12,3.882495e-11,1.076180e-06,6.841110e-29,0.0,1.000000
75%,298.15,0.700000,0.0,5.670000e-08,2.430000e-08,1.000000e-08,2.125764e-08,4.252500e-08,0.0,0.0,3.085270e-07,1.356510e-11,5.413690e-11,1.533910e-06,8.350190e-29,0.0,1.000000
max,298.15,0.850000,0.0,8.000000e-08,4.000000e-08,2.250000e-08,4.000000e-08,6.000000e-08,0.0,0.0,1.109770e-06,1.670600e-10,4.010640e-10,2.744790e-06,9.845690e-29,0.0,1.000000


In [31]:
factor = pd.factorize(df['phase'])
df.phase = factor[0]
definitions = factor[1]

In [32]:
X = df.drop(columns=['Temp(K)', 'N_H+(mol)', 'N_OH-(mol)', 'N_NH3-(mol)', 'Water Content(mol)', 
                     'P_HNO3(atm)', 'P_HCL(atm)', 'P_NH3(atm)', 'P_H2SO4(atm)', 'P_HBr(atm)', 'contains_solid', 'phase'])
y = df['phase']

In [33]:
#df["Water Content(mol)"] = np.log(df["Water Content(mol)"])
scalers = {}
for col in X.columns:
    mean = X[col].mean()
    std = X[col].std(ddof=0)
    X[col] = (X[col] - mean) / std
    scalers[col] = (mean, std)
    '''
    maximum = max(df[col])
    minimum = min(df[col])
    df[col] = (df[col] - minimum) / (maximum - minimum)
    scalers[col] = (minimum, maximum)
    '''

'''
#Standardize water content
water_content_mean = target.mean()
water_content_std = target.std(ddof=0)
df["Water Content(mol)"] = (df["Water Content(mol)"] - water_content_mean) / water_content_std
'''
X.describe()

,RH,N_NH4+(mol),N_Na+(mol),N_SO42-(mol),N_NO3-(mol),N_Cl-(mol)
count,2.480000e+04,2.480000e+04,2.480000e+04,2.480000e+04,2.480000e+04,2.480000e+04
mean,-9.852334e-16,-1.346593e-17,1.615912e-16,2.246232e-16,1.146037e-17,6.861895e-17
std,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00,1.000020e+00
min,-1.772316e+00,-1.607420e-01,-2.465838e+00,-1.631763e+00,-1.337814e-01,-3.261753e-01
25%,-6.646185e-01,-1.266932e-01,-7.336939e-01,-7.395670e-01,-1.085876e-01,-1.295712e-01
50%,2.769244e-02,-9.256570e-02,-1.664835e-01,-1.342873e-01,-8.554762e-02,-7.189606e-02
75%,7.200034e-01,-4.945400e-02,6.382907e-01,6.224487e-01,-5.576406e-02,6.775440e-03
max,1.550777e+00,1.639454e+01,2.643839e+00,3.440213e+00,2.152596e+01,2.316241e+01


In [34]:
#split for train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  shuffle=True)

In [35]:
# Fitting Random Forest Classification to the Training set
rf_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy')

In [9]:
print(list(zip(X, rf_classifier.feature_importances_)))
joblib.dump(rf_classifier, 'saved_models/rf_phase_classifier.pkl') 

[('RH', 0.9062775949405316), ('N_NH4+(mol)', 0.01281999765760212), ('N_Na+(mol)', 0.013604433829746385), ('N_SO42-(mol)', 0.022955010948180823), ('N_NO3-(mol)', 0.02442283183869705), ('N_Cl-(mol)', 0.01992013078524196)]


['saved_models/rf_phase_classifier.pkl']